In [203]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import ssl
import json
import re
ssl._create_default_https_context = ssl._create_unverified_context

In [204]:
sitemap = pd.read_xml('https://www.lalibre.be/arc/outboundfeeds/sitemap-news/?outputType=xml')
sitemap.head()

,loc,lastmod,news,image
0,https://www.lalibre.be/culture/musique/2023/08...,2023-08-01T14:03:58.552Z,NaN,NaN
1,https://www.lalibre.be/etudiant/vie-etudiante/...,2023-08-02T10:13:45.351Z,NaN,NaN
2,https://www.lalibre.be/etudiant/2023/08/02/cha...,2023-08-02T10:06:38.920Z,NaN,NaN
3,https://www.lalibre.be/international/europe/20...,2023-08-02T10:03:02.662Z,NaN,NaN
4,https://www.lalibre.be/regions/liege/2023/08/0...,2023-08-02T09:58:07.575Z,NaN,NaN


In [205]:
df = sitemap.drop(["news", "image"], axis=1)
df.rename(columns={"loc": "source_url", "lastmod": "last_modified_date"}, inplace=True)
df["last_modified_date"].replace({r"T.+": ""}, inplace=True, regex=True)
df.head()

,source_url,last_modified_date
0,https://www.lalibre.be/culture/musique/2023/08...,2023-08-01
1,https://www.lalibre.be/etudiant/vie-etudiante/...,2023-08-02
2,https://www.lalibre.be/etudiant/2023/08/02/cha...,2023-08-02
3,https://www.lalibre.be/international/europe/20...,2023-08-02
4,https://www.lalibre.be/regions/liege/2023/08/0...,2023-08-02


In [206]:
urls = df['source_url']
urls.head

<bound method NDFrame.head of 0     https://www.lalibre.be/culture/musique/2023/08...
1     https://www.lalibre.be/etudiant/vie-etudiante/...
2     https://www.lalibre.be/etudiant/2023/08/02/cha...
3     https://www.lalibre.be/international/europe/20...
4     https://www.lalibre.be/regions/liege/2023/08/0...
                            ...                        
95    https://www.lalibre.be/dernieres-depeches/2023...
96    https://www.lalibre.be/dernieres-depeches/2023...
97    https://www.lalibre.be/economie/placements/202...
98    https://www.lalibre.be/international/europe/20...
99    https://www.lalibre.be/international/europe/20...
Name: source_url, Length: 100, dtype: object>

In [207]:
article_one = urls[0]
print(article_one)

https://www.lalibre.be/culture/musique/2023/08/01/trop-special-trop-talentueux-un-des-acteurs-les-plus-prometteurs-dhollywood-angus-cloud-est-decede-une-semaine-apres-son-pere-JPKARWVD4NEX7CIVGZXLM3RGAU/


In [208]:
response = requests.get(article_one)
response.text

'<!DOCTYPE html><html lang="fr"><head><script>\n\tfunction isItIE() {\n\t  user_agent = navigator.userAgent;\n\t  var is_it_ie = user_agent.indexOf("MSIE ") > -1 || user_agent.indexOf("Trident/") > -1;\n\t  return is_it_ie; \n\t}\n\tif (isItIE()){\n\t    alert(\'Pour accéder à ce site, veuillez utiliser un navigateur plus récent.\');\n\t}\n</script><meta name="viewport" content="width=device-width,initial-scale=1,minimum-scale=1.0,maximum-scale=1.0,user-scalable=no"/><link rel="preload" as="image" imagesrcset="https://www.lalibre.be/resizer/aSILGXI_-hJuE2RuEA3Q-CwfOgc=/480x320/filters:format(jpeg):focal(2935x2267:2945x2257)/cloudfront-eu-central-1.images.arcpublishing.com/ipmgroup/YPCNEJXZ2BHEFLYELZUMIB2RZ4.jpg 480w,https://www.lalibre.be/resizer/8Jv9yMRJ-j0a-5QYbDAFRA0WodI=/768x512/filters:format(jpeg):focal(2935x2267:2945x2257)/cloudfront-eu-central-1.images.arcpublishing.com/ipmgroup/YPCNEJXZ2BHEFLYELZUMIB2RZ4.jpg 768w,https://www.lalibre.be/resizer/ZKy6EhK70VsY3Xc7s3VsbWSnKOM=/1200

In [209]:
soup = bs(response.content, "html.parser")

# Title

In [210]:
def find_article_title(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    title = soup.find("h1").text
    article_title = title.strip()
    return article_title

In [211]:
print(find_article_title(article_one))

"Trop spécial, trop talentueux": un des acteurs les plus prometteurs d’Hollywood, Angus Cloud, est décédé une semaine après son père


# Text

In [212]:
def find_article_text(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    paragraphs = [p.text for p in soup.select('p[class~="ap-StoryText"]')]
    article_text = "".join(paragraphs)
    return article_text

In [213]:
print(find_article_text(article_one))

Le destin est parfois cruel. A Hollywood, certains comparaient Angus Cloud à James Dean, de par son côté rebelle, son look glamour et ce petit quelque chose d’animal dans ses interprétations. Son destin semblait tracé dans les étoiles, avec un nom destiné à figurer tout en haut des affiches. D’ailleurs, l’an dernier, pour son numéro consacré aux jeunes acteurs les plus prometteurs du 7e art américain, Variety l’avait choisi pour occuper toute la couverture du magazine. Malheureusement, sa filmographie contiendra encore moins de lignes que celle de son idole : il a été retrouvé mort ce mardi 1er août dans son domicile à Oakland. Il n’avait que 25 ans.”C’est le cœur lourd que nous avons dû dire au revoir à un être humain incroyable aujourd’hui, a fait savoir sa famille par communiqué. En tant qu’artiste, ami, frère et fils, Angus était spécial pour nous tous à bien des égards. La semaine dernière, il a enterré son père et a lutté intensément contre cette perte. Notre seul réconfort est d

# Date

In [214]:
def find_published_date(url:str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    date = [p.text for p in soup.find('p')]
    data = " ".join(date)
    date_pattern = r"\d{2}-\d{2}-\d{4}" 
    date_match = re.search(date_pattern, data)
    date = date_match.group()
    day, month, year = date.split("-")
    date= f"{year}-{month}-{day}"  
    return date

In [215]:
print(find_published_date(article_one))

2023-08-01


In [216]:
df['published_date'] = df['source_url'].apply(find_published_date)
df.head()

,source_url,last_modified_date,published_date
0,https://www.lalibre.be/culture/musique/2023/08...,2023-08-01,2023-08-01
1,https://www.lalibre.be/etudiant/vie-etudiante/...,2023-08-02,2023-08-02
2,https://www.lalibre.be/etudiant/2023/08/02/cha...,2023-08-02,2023-08-02
3,https://www.lalibre.be/international/europe/20...,2023-08-02,2023-08-02
4,https://www.lalibre.be/regions/liege/2023/08/0...,2023-08-02,2023-08-02


In [217]:
df['article_title'] = df['source_url'].apply(find_article_title)
df

,source_url,last_modified_date,published_date,article_title
0,https://www.lalibre.be/culture/musique/2023/08...,2023-08-01,2023-08-01,"""Trop spécial, trop talentueux"": un des acteur..."
1,https://www.lalibre.be/etudiant/vie-etudiante/...,2023-08-02,2023-08-02,Sarah Chaâri offre une médaille de bronze à la...
2,https://www.lalibre.be/etudiant/2023/08/02/cha...,2023-08-02,2023-08-02,Charleroi: le campus universitaire et son quar...
3,https://www.lalibre.be/international/europe/20...,2023-08-02,2023-08-02,Les images de l'arrivée du pape à Lisbonne pou...
4,https://www.lalibre.be/regions/liege/2023/08/0...,2023-08-02,2023-08-02,Le tunnel de Cointe sera fermé dans les deux s...
...,...,...,...,...
95,https://www.lalibre.be/dernieres-depeches/2023...,2023-08-01,2023-08-01,"Macron à Brégançon, pause estivale ""studieuse""..."
96,https://www.lalibre.be/dernieres-depeches/2023...,2023-08-01,2023-08-01,La faucille et le marteau retirés d'une statue...
97,https://www.lalibre.be/economie/placements/202...,2023-08-01,2023-08-01,Mauvaise aura sur les Bourses européennes ce m...
98,https://www.lalibre.be/international/europe/20...,2023-08-01,2023-08-01,Drame en France : un enfant de cinq ans se noi...


In [218]:
df['article_text'] = df['source_url'].apply(find_article_text)
df = df.loc[:, ['source_url', 'article_title', 'article_text', 'published_date', 'last_modified_date']]
df.to_csv('lalibre_articles.csv')
df.head()

,source_url,article_title,article_text,published_date,last_modified_date
0,https://www.lalibre.be/culture/musique/2023/08...,"""Trop spécial, trop talentueux"": un des acteur...","Le destin est parfois cruel. A Hollywood, cert...",2023-08-01,2023-08-01
1,https://www.lalibre.be/etudiant/vie-etudiante/...,Sarah Chaâri offre une médaille de bronze à la...,"En tout, cinq Belges ont fait le déplacement :...",2023-08-02,2023-08-02
2,https://www.lalibre.be/etudiant/2023/08/02/cha...,Charleroi: le campus universitaire et son quar...,La Ville de Charleroi a prévu différents événe...,2023-08-02,2023-08-02
3,https://www.lalibre.be/international/europe/20...,Les images de l'arrivée du pape à Lisbonne pou...,"""Je reviendrai (de ce voyage) rajeuni"", a-t-il...",2023-08-02,2023-08-02
4,https://www.lalibre.be/regions/liege/2023/08/0...,Le tunnel de Cointe sera fermé dans les deux s...,Alors que la liaison autoroutière E25-E40/A602...,2023-08-02,2023-08-02
